有多种方法可以加快模型的收敛速度：
* gpu/tpu 训练
* 混合精度（16 位）训练
* 控制训练时期
* 控制验证频率
* 限制数据集大小
* 将数据预加载到 ram
* 模型切换
* 将 grads 设置为 none
* 要避免的事情

# GPU/TPU 训练

何时使用：只要有可能！

使用 Lightning，在 GPU、TPU 或多个节点上运行是一个简单的标志切换。

## GPU训练

Lightning 支持多种插件，进一步加速分布式 GPU 训练。 最为显着地：
* `DDPPlugin`
* `DDPShardedPlugin`
* `DeepSpeedPlugin`

In [ ]:
# 在 1 gpu 上运行
trainer = Trainer(gpus=1)

# 在 8 gpus 上训练，使用 DDP 插件
trainer = Trainer(gpus=8, accelerator="ddp")

# 在跨节点的多个 GPU 上训练（总共使用 8 个 GPU）
trainer = Trainer(gpus=2, num_nodes=4)

### GPU 训练加速技巧

在单台或多台 GPU 机器上训练时，Lightning 提供了一系列高级优化来提高吞吐量、内存效率和模型扩展。 有关更多详细信息，请参阅高级 GPU 优化训练。

### 比 DP 更喜欢 DDP

`DataParallelPlugin` 为每个批次执行三个 GPU 传输：
1. 将模型复制到设备。
2. 将数据复制到设备。
3. 将每个设备的输出复制回主设备。
而 `DDPPlugin` 只执行 1 次传输来同步梯度，使得 `DDP` 比 `DP` 快得多。

使用 `DDP` 插件时，设置 `find_unused_parameters=False`

默认情况下，出于过去观察到的兼容性原因，我们已将 `find_unused_parameters` 设置为 `True`（有关更多详细信息，请参阅讨论）。 默认情况下，这会影响性能，并且在大多数情况下可以禁用。

> 它适用于所有支持 find_unused_parameters 作为输入的 DDP 插件。

In [ ]:
from pytorch_lightning.plugins import DDPPlugin

trainer = pl.Trainer(
    gpus=2,
    plugins=DDPPlugin(find_unused_parameters=False),
)

In [ ]:
from pytorch_lightning.plugins import DDPSpawnPlugin

trainer = pl.Trainer(
    gpus=2,
    plugins=DDPSpawnPlugin(find_unused_parameters=False),
)

### 在多节点集群上使用DDP时，设置NCCL参数

NCCL 是 NVIDIA 集体通信库，PyTorch 在后台使用它来处理节点和 GPU 之间的通信。 在调整 NCCL 参数时，在加速方面有报告的好处，如本期所示。 在这个问题中，我们看到训练 Transformer XLM-RoBERTa 时速度提高了 30%，使用 Detectron2 训练时速度提高了 15%。

NCCL 参数可以通过环境变量进行调整。

> AWS 和 GCP 已经在它们的集群上为这些设置了默认值。 这对于自定义集群设置通常很有用。

* NCCL_NSOCKS_PERTHREAD
* NCCL_SOCKET_NTHREADS
* NCCL_MIN_NCHANNELS

In [ ]:
export NCCL_NSOCKS_PERTHREAD=4
export NCCL_SOCKET_NTHREADS=2

### 数据加载器

构建 DataLoader 时，设置 `num_workers > 0` 和 `pin_memory=True`（仅适用于 GPU）。

In [ ]:
Dataloader(dataset, num_workers=8, pin_memory=True)

### num_workers

在 `num_workers` 中指定多少工人的问题很棘手。 以下是一些参考文献 [1] 和我们的建议的摘要：
* `num_workers=0` 意味着只有主进程会加载批次（这可能是一个瓶颈）。
* `num_workers=1` 意味着只有一个工人（不是主进程）会加载数据，但它仍然会很慢。
* `num_workers` 取决于批量大小和您的机器。
* 一般开始的地方是将 `num_workers` 设置为等于该机器上的 CPU 内核数。 您可以使用 `os.cpu_count()` 在 python 中获取 CPU 内核数，但请注意，根据您的批次大小，您可能会溢出 RAM 内存。

> 增加 num_workers 也会增加你的 CPU 内存消耗。

最好的办法是缓慢增加 num_workers 并在您看到训练速度没有更多提高时停止。

### Spawn

当使`·Accelerator=ddp_spawn` 或在TPU 上训练时，使用多个GPU/TPU 内核的方式是在后台调用`.spawn()`。 问题是 PyTorch 在使用 `.spawn()` 时存在 num_workers > 0 的问题。 出于这个原因，我们建议您使用加速器 = ddp 以便您可以增加 num_workers，但是您的脚本必须是可调用的，如下所示：

In [ ]:
python my_program.py

## TPU培训

您可以将 `tpu_cores` 训练器标志设置为 1 或 8 核。

In [ ]:
# train on 1 TPU core
trainer = Trainer(tpu_cores=1)

# train on 8 TPU cores
trainer = Trainer(tpu_cores=8)

In [ ]:
要在超过 8 个内核（即：一个 POD）上进行训练，请使用 xla_dist 脚本提交此脚本。

例子：

In [ ]:
python -m torch_xla.distributed.xla_dist
--tpu=$TPU_POD_NAME
--conda-env=torch-xla-nightly
--env=XLA_USE_BF16=1
-- python your_trainer_file.py

# 混合精度（16 位）训练

在以下情况下使用：
* 您想针对 GPU 上的内存使用进行优化。
* 您有一个支持 16 位精度的 GPU（NVIDIA pascal 架构或更新）。
* 您的优化算法 (training_step) 在数值上是稳定的。
* 你想成为实验室里最酷的人 :p

混合精度结合使用 32 位和 16 位浮点来减少模型训练期间的内存占用，从而提高性能，在现代 GPU 上实现 3 倍的加速。

Lightning 提供针对 GPU 和 CPU 的混合精度训练，以及针对 TPU 的 bfloat16 混合精度训练。

In [ ]:
# 16-bit precision
trainer = Trainer(precision=16, gpus=4)

# 控制训练epoch

**何时使用**：您运行超参数搜索以找到合适的初始参数并希望节省时间、成本（金钱）或功耗（环境）。 它可以让您更具成本效益，并同时运行更多的实验。

您可以使用 Trainer 标志来强制训练最少的 epoch 数或限制为最大的 epoch 数。 使用 min_epochs 和 max_epochs Trainer 标志设置要运行的纪元数。

In [ ]:
# DEFAULT
trainer = Trainer(min_epochs=1, max_epochs=1000)

如果运行基于迭代的训练，即无限/可迭代数据加载器，您还可以使用 min_steps 和 max_steps 标志控制步数：

In [ ]:
trainer = Trainer(max_steps=1000)

trainer = Trainer(min_steps=100)

您还可以根据训练时间中断训练：

In [ ]:
# 训练 12 小时后或达到 10 个时期时停止（字符串）
trainer = Trainer(max_time="00:12:00:00", max_epochs=10)

# 1天5小时后停止（字典）
trainer = Trainer(max_time={"days": 1, "hours": 5})

在我们的 Trainer flags指南中了解更多信息。

# 控制验证频率

## 每 n 个时期检查一次验证

在以下情况下使用：您有一个小数据集，并希望运行较少的验证检查。

您可以使用 `check_val_every_n_epoch` Trainer 标志将验证检查限制为仅每 n 次运行一次。

In [ ]:
# DEFAULT
trainer = Trainer(check_val_every_n_epoch=1)

## 在 1 个训练epoch内设置验证检查频率

何时使用：您有一个大型训练数据集，并且想要运行中期验证检查。

对于大型数据集，通常需要在训练循环中多次检查验证。 通常在 1 个训练时期内传递一个浮点数来检查它。 传入一个 int k 来检查每 k 个训练批次。 如果使用 IterableDataset，则必须使用 int。

In [ ]:
# DEFAULT
trainer = Trainer(val_check_interval=0.95)

# 每 0.25 次检查一次
trainer = Trainer(val_check_interval=0.25)

# 每 100 个训练批次检查一次（即：对于`IterableDatasets` 或固定频率）
trainer = Trainer(val_check_interval=100)

# 限制数据集大小

## 使用数据子集进行训练、验证和测试

用于：调试或运行大型数据集。

如果您不想检查 100% 的训练/验证/测试集，请设置这些标志：

In [ ]:
# DEFAULT
trainer = Trainer(limit_train_batches=1.0, limit_val_batches=1.0, limit_test_batches=1.0)

# 仅检查 10%、20%、30%，分别用于训练、验证和测试集
trainer = Trainer(limit_train_batches=0.1, limit_val_batches=0.2, limit_test_batches=0.3)

如果您还将 `shuffle=True` 传递给数据加载器，则每个时期将使用数据集的不同随机子集； 否则相同的子集将用于所有时期。

> * `overfit_batches` > 0，`limit_train_batches`、`limit_val_batches` 和 `limit_test_batches` 将被 `overfit_batches` 覆盖。
> * `fast_dev_run=True`，`limit_val_batches` 将被忽略。

# 将数据预加载到 RAM

何时使用：您需要一次访问数据集中的所有样本。

当您的训练或预处理需要对整个数据集执行许多操作时，如果有足够的空间，有时将所有数据存储在 RAM 中可能会有所帮助。 但是，在训练脚本的开头加载所有数据的缺点是可能需要很长时间，因此会减慢开发过程。 另一个缺点是在多处理（例如 DDP）中，数据会在每个进程中被复制。 可以通过提前将数据复制到 RAM 中来克服这些问题。 大多数基于 UNIX 的操作系统通过通常名为 /dev/shm 的挂载点提供对 tmpfs 的直接访问。
1. 必要时增加共享内存。 请参阅您的操作系统的文档如何执行此操作。
2. 将训练数据复制到共享内存：
```
cp -r /path/to/data/on/disk /dev/shm/
```
3. 请参阅脚本或命令行参数中的新数据根：
```
datamodule = MyDataModule(data_root="/dev/shm/my_data")
```

# 模型切换

用于：在分布式设置中使用多个优化器执行梯度累积。

下面是它的作用的解释：
* 将当前优化器视为 A，将所有其他优化器视为 B。
* 切换意味着从 B 到 A 的所有参数都将它们的 `requires_grad` 属性设置为 `False`。
* 退出上下文管理器时，它们的原始状态将恢复。

在进行梯度累积时，不需要在累积阶段进行梯度同步。 将 `sync_grad` 设置为 `False` 将阻止此同步并提高您的训练速度。

`LightningOptimizer` 为高级用户提供了一个 `toggle_model()` 函数作为 `contextlib.contextmanager()`。

以下是高级用例的示例：

In [ ]:
# 每 2 个批次进行梯度累积并针对多个 GPU 进行优化的 GAN 场景。
class SimpleGAN(LightningModule):
    def __init__(self):
        super().__init__()
        self.automatic_optimization = False

    def training_step(self, batch, batch_idx):
        # 实现遵循 PyTorch 教程：
        # https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
        g_opt, d_opt = self.optimizers()

        X, _ = batch
        X.requires_grad = True
        batch_size = X.shape[0]

        real_label = torch.ones((batch_size, 1), device=self.device)
        fake_label = torch.zeros((batch_size, 1), device=self.device)

        # 在累积结束或 epoch 结束时同步和清除梯度。
        is_last_batch_to_accumulate = (batch_idx + 1) % 2 == 0 or self.trainer.is_last_batch

        g_X = self.sample_G(batch_size)

        ##########################
        # Optimize Discriminator #
        ##########################
        with d_opt.toggle_model(sync_grad=is_last_batch_to_accumulate):
            d_x = self.D(X)
            errD_real = self.criterion(d_x, real_label)

            d_z = self.D(g_X.detach())
            errD_fake = self.criterion(d_z, fake_label)

            errD = errD_real + errD_fake

            self.manual_backward(errD)
            if is_last_batch_to_accumulate:
                d_opt.step()
                d_opt.zero_grad()

        ######################
        # Optimize Generator #
        ######################
        with g_opt.toggle_model(sync_grad=is_last_batch_to_accumulate):
            d_z = self.D(g_X)
            errG = self.criterion(d_z, real_label)

            self.manual_backward(errG)
            if is_last_batch_to_accumulate:
                g_opt.step()
                g_opt.zero_grad()

        self.log_dict({"g_loss": errG, "d_loss": errD}, prog_bar=True)

# 将Grads设置为无

为了适度提高性能，您可以覆盖 optimizer_zero_grad()。

有关此技术优缺点的更详细说明，请阅读 PyTorch 团队的 zero_grad() 文档。

In [ ]:
class Model(LightningModule):
    def optimizer_zero_grad(self, epoch, batch_idx, optimizer, optimizer_idx):
        optimizer.zero_grad(set_to_none=True)

# 需要避免的事情

`.item(), .numpy(), .cpu()`

不要在代码中的任何地方调用 `.item()`。 使用 `.detach()` 来移除连接的图调用。 为此，Lightning 需要非常小心地进行优化。

# empty_cache()

不要不必要地调用这个！ 每次调用此命令时，所有 GPU 都必须等待同步。

# 将张量传输到设备
LightningModules知道它们在什么设备上！ 直接在设备上构造张量，避免CPU->设备传输。

In [ ]:
# bad
t = torch.rand(2, 2).cuda()

# good (self is LightningModule)
t = torch.rand(2, 2, device=self.device)

对于需要作为模型属性的张量，最佳做法是在模块的 `__init__` 方法中将它们注册为缓冲区：

In [ ]:
# bad
self.t = torch.rand(2, 2, device=self.device)

# good
self.register_buffer("t", torch.rand(2, 2))